## Setup

**&raquo; Create a `sparql_env`.**

In [ ]:
Boyle.mk("sparql_env")

All dependencies up to date


{:ok, ["rdf_env", "sparql_env"]}

**&raquo; Activate the environment.**

In [ ]:
Boyle.activate("sparql_env")

All dependencies up to date


:ok

**&raquo; Install `sparql_client` dependencies.**

In [ ]:
Boyle.install( {:sparql_client, "~> 0.2.1"})

Resolving Hex dependencies...
Dependency resolution completed:
Unchanged:
  content_type 0.1.0
  decimal 1.5.0
  elixir_uuid 1.2.0
  jason 1.1.2
  json_ld 0.3.0
  mime 1.3.0
  nimble_csv 0.4.0
  rdf 0.5.1
  sparql 0.2.5
  sparql_client 0.2.1
  sweet_xml 0.6.5
  tesla 1.1.0
All dependencies up to date


:ok

In [ ]:
exports SPARQL

execute_query/2                   execute_query/3                   query/1                           
query/2                           result_format/1                   result_format_by_extension/1      
result_format_by_media_type/1     result_formats/0                  


In [ ]:
exports SPARQL.Client

__adapter__/0               __middleware__/0            default_accept_header/1     
delete/1                    delete/2                    delete/3                    
delete!/1                   delete!/2                   delete!/3                   
get/1                       get/2                       get/3                       
get!/1                      get!/2                      get!/3                      
head/1                      head/2                      head/3                      
head!/1                     head!/2                     head!/3                     
options/1                   options/2                   options/3                   
options!/1                  options!/2                  options!/3                  
patch/2                     patch/3                     patch/4                     
patch!/2                    patch!/3                    patch!/4                    
post/2                      post/3                      post/4   

**&raquo; Let's set up a simple SPARQL query.**

In [ ]:
query = "select * where {?s ?p ?o} limit 5"

**&raquo; And choose a SPARQL endpoint.**

In [ ]:
service = "http://dbpedia.org/sparql"

**&raquo; First query try gives error.**

In [ ]:
SPARQL.Client.query(query, service)

**&raquo; Let's check out the documentation.**

In [ ]:
h SPARQL.Client.query

**&raquo; Second try succeeds – after setting :request_method (and :protocol_version).**

In [ ]:
SPARQL.Client.query(query, service, request_method: :get, protocol_version: "1.1")

**&raquo; Is our `TestQuery` module loaded?**

In [ ]:
exports TestQuery

**&raquo; That'll be a no, then.**

In [ ]:
import_file("/Users/tony/Projects/local/elixir/examples/test_query/lib/test_query.ex")

**&raquo; Rats!**

Turns out that our call `:code.priv_dir(:test_query)}` is failing.

Let's just hardwire that for now and try again.

In [ ]:
import_file("/Users/tony/Projects/local/elixir/examples/test_query/lib/test_query.ex")

In [ ]:
exports TestQuery

data/0      query/0     query/1     query/2     


Success!

Now let's import the `TestQuery.Client` module.

In [17]:
import_file("/Users/tony/Projects/local/elixir/examples/test_query/lib/test_query/client.ex")

Protocol.UndefinedError: 1

Same problem.

In [ ]:
import_file("/Users/tony/Projects/local/elixir/examples/test_query/lib/test_query/client.ex")

{:module, TestQuery.Client, <<70, 79, 82, 49, 0, 0, 25, 168, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 195, 0, 0, 0, 69, 23, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 46, 67, 108, 105, 101, 110, ...>>, {:_read_tuple, 1}}

In [ ]:
exports TestQuery.Client

get_query/0       get_service/0     hello/0           read_table/1      rquery/0          
rquery/1          rquery/2          rquery_all/0      


In [ ]:
result

%SPARQL.Query.Result{results: [%{"o" => ~L"Hello World"en, "p" => ~I<http://www.w3.org/2000/01/rdf-schema#label>, "s" => ~I<http://dbpedia.org/resource/Hello_World>}], variables: ["s", "p", "o"]}

In [ ]:
o = SPARQL.Query.Result.get(result, :o) |> List.first

~L"Hello World"en

In [32]:
query

File.Error: 1

In [32]:
i query

File.Error: 1

In [ ]:
o

[~L"Hello World"en]

In [ ]:
o.value

"Hello World"

Now let's add in some `TestSuper.Client`.

In [ ]:
rq = File.read!("/Users/tony/Projects/local/elixir/examples/test_super/priv/queries/dbpedia_query.rq")

"prefix dbo: <http://dbpedia.org/ontology/>\nprefix foaf: <http://xmlns.com/foaf/0.1/>\nprefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n\nselect *\nwhere {\n  bind (12345 as ?id)\n  ?s dbo:wikiPageID ?id .\n  optional { ?s foaf:isPrimaryTopicOf ?topic }\n  optional { ?s rdfs:label ?label }\n  filter (langMatches(lang(?label), \"en\"))\n} limit 1\n"

In [ ]:
IO.puts rq

prefix dbo: <http://dbpedia.org/ontology/>
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select *
where {
  bind (12345 as ?id)
  ?s dbo:wikiPageID ?id .
  optional { ?s foaf:isPrimaryTopicOf ?topic }
  optional { ?s rdfs:label ?label }
  filter (langMatches(lang(?label), "en"))
} limit 1



:ok

In [ ]:
exports 

In [ ]:
exports SPARQL.Query.Result

__struct__/0          __struct__/1          add_identity/1        append/2              
get/2                 new/1                 new/2                 remove_identity/1     


In [ ]:
 objects = (result |> SPARQL.Query.Result.get(:o) )

[~I<https://twitter.com/bgmarx>, ~I<https://twitter.com/josevalim>, ~I<https://twitter.com/redrapids>, %RDF.Literal{value: ~D[2018-03-14], datatype: ~I<http://www.w3.org/2001/XMLSchema#date>}, ~L"Paper", ~I<https://pragprog.com/>, ~L"Adopting Elixir"en, ~I<http://purl.org/ontology/bibo/Book>]

In [ ]:
IO.inspect objects

[~I<https://twitter.com/bgmarx>, ~I<https://twitter.com/josevalim>,
 ~I<https://twitter.com/redrapids>,
 %RDF.Literal{value: ~D[2018-03-14], datatype: ~I<http://www.w3.org/2001/XMLSchema#date>},
 ~L"Paper", ~I<https://pragprog.com/>, ~L"Adopting Elixir"en,
 ~I<http://purl.org/ontology/bibo/Book>]


[~I<https://twitter.com/bgmarx>, ~I<https://twitter.com/josevalim>, ~I<https://twitter.com/redrapids>, %RDF.Literal{value: ~D[2018-03-14], datatype: ~I<http://www.w3.org/2001/XMLSchema#date>}, ~L"Paper", ~I<https://pragprog.com/>, ~L"Adopting Elixir"en, ~I<http://purl.org/ontology/bibo/Book>]

In [ ]:
objects

[~I<https://twitter.com/bgmarx>, ~I<https://twitter.com/josevalim>, ~I<https://twitter.com/redrapids>, %RDF.Literal{value: ~D[2018-03-14], datatype: ~I<http://www.w3.org/2001/XMLSchema#date>}, ~L"Paper", ~I<https://pragprog.com/>, ~L"Adopting Elixir"en, ~I<http://purl.org/ontology/bibo/Book>]

In [ ]:
objects

[~I<https://twitter.com/bgmarx>, ~I<https://twitter.com/josevalim>, ~I<https://twitter.com/redrapids>, %RDF.Literal{value: ~D[2018-03-14], datatype: ~I<http://www.w3.org/2001/XMLSchema#date>}, ~L"Paper", ~I<https://pragprog.com/>, ~L"Adopting Elixir"en, ~I<http://purl.org/ontology/bibo/Book>]

In [ ]:
IO.puts objects

In [1]:
objects

CompileError: 1

In [1]:
result

CompileError: 1

In [ ]:
objects |> IO.puts

In [1]:
objects

CompileError: 1

In [1]:
query

CompileError: 1

In [ ]:
exports Application

__using__/1                app_dir/1                  app_dir/2                  
delete_env/2               delete_env/3               ensure_all_started/1       
ensure_all_started/2       ensure_started/1           ensure_started/2           
fetch_env/2                fetch_env!/2               format_error/1             
get_all_env/1              get_application/1          get_env/2                  
get_env/3                  load/1                     loaded_applications/0      
put_env/3                  put_env/4                  spec/1                     
spec/2                     start/1                    start/2                    
started_applications/0     started_applications/1     stop/1                     
unload/1                   


In [ ]:
Application.loaded_applications

[{:sqlite_ecto, 'SQLite3 adapter for Ecto', '1.1.0'}, {:poolboy, 'A hunky Erlang worker pool factory', '1.5.1'}, {:stdlib, 'ERTS  CXC 138 10', '3.5.1'}, {:iex, 'iex', '1.7.3'}, {:inets, 'INETS  CXC 138 49', '7.0.1'}, {:hex, 'hex', '0.18.1'}, {:elixir, 'elixir', '1.7.3'}, {:logger, 'logger', '1.7.3'}, {:ecto, 'Ecto is a domain specific language for writing queries and interacting with databases in Elixir.\n', '1.1.9'}, {:kernel, 'ERTS  CXC 138 10', '6.0'}, {:public_key, 'Public key infrastructure', '1.6.1'}, {:mix, 'mix', '1.7.3'}, {:ssl, 'Erlang/OTP SSL application', '9.0.1'}, {:uuid, 'Erlang UUID Implementation', '1.7.2'}, {:poison, 'The fastest JSON library for Elixir', '1.5.2'}, {:quickrand, 'Quick Random Number Generation', '1.7.2'}, {:compiler, 'ERTS  CXC 138 10', '7.2.3'}, {:erlzmq, 'Erlang ZeroMQ Driver', '3.0'}, {:asn1, 'The Erlang ASN1 compiler version 5.0.6', '5.0.6'}, {:crypto, 'CRYPTO', '4.3.2'}, {:ielixir, 'Jupyter\'s kernel for Elixir programming language\n', '0.9.14'}, {

In [1]:
 result

CompileError: 1

In [ ]:
Application.loaded_applications

[{:sqlite_ecto, 'SQLite3 adapter for Ecto', '1.1.0'}, {:poolboy, 'A hunky Erlang worker pool factory', '1.5.1'}, {:stdlib, 'ERTS  CXC 138 10', '3.5.1'}, {:iex, 'iex', '1.7.3'}, {:inets, 'INETS  CXC 138 49', '7.0.1'}, {:hex, 'hex', '0.18.1'}, {:elixir, 'elixir', '1.7.3'}, {:logger, 'logger', '1.7.3'}, {:ecto, 'Ecto is a domain specific language for writing queries and interacting with databases in Elixir.\n', '1.1.9'}, {:kernel, 'ERTS  CXC 138 10', '6.0'}, {:public_key, 'Public key infrastructure', '1.6.1'}, {:mix, 'mix', '1.7.3'}, {:ssl, 'Erlang/OTP SSL application', '9.0.1'}, {:uuid, 'Erlang UUID Implementation', '1.7.2'}, {:compiler, 'ERTS  CXC 138 10', '7.2.3'}, {:poison, 'The fastest JSON library for Elixir', '1.5.2'}, {:quickrand, 'Quick Random Number Generation', '1.7.2'}, {:erlzmq, 'Erlang ZeroMQ Driver', '3.0'}, {:asn1, 'The Erlang ASN1 compiler version 5.0.6', '5.0.6'}, {:crypto, 'CRYPTO', '4.3.2'}, {:ielixir, 'Jupyter\'s kernel for Elixir programming language\n', '0.9.14'}, {

In [ ]:
exports Application

__using__/1                app_dir/1                  app_dir/2                  
delete_env/2               delete_env/3               ensure_all_started/1       
ensure_all_started/2       ensure_started/1           ensure_started/2           
fetch_env/2                fetch_env!/2               format_error/1             
get_all_env/1              get_application/1          get_env/2                  
get_env/3                  load/1                     loaded_applications/0      
put_env/3                  put_env/4                  spec/1                     
spec/2                     start/1                    start/2                    
started_applications/0     started_applications/1     stop/1                     
unload/1                   


In [ ]:
Application.load(:test_query)

{:error, {'no such file or directory', 'test_query.app'}}

In [11]:
Application.load("/Users/tony/Projects/local/elixir/examples/test_query/_build/dev/lib/test_query/ebin/test_query.app")

FunctionClauseError: 1

In [ ]:
Application.

In [ ]:
h Application.load


                                 def load(app)                                  

  @spec load(app()) :: :ok | {:error, term()}

Loads the given app.

In order to be loaded, an .app file must be in the load paths. All
:included_applications will also be loaded.

Loading the application does not start it nor load its modules, but it does
load its environment.



In [ ]:
exports Boyle


activate/1             active_env_name/0      child_spec/1           code_change/3          
deactivate/0           environment_path/0     freeze/0               handle_call/3          
handle_cast/2          handle_info/2          init/1                 install/1              
list/0                 mk/1                   paths/0                reinstall/0            
rm/1                   start_link/1           state/0                terminate/2            
